In [1]:
!pip install -q groq speechbrain pydub scikit-learn

/Users/vishakhabhayan/.zshenv:.:1: no such file or directory: /Users/vishakhabhayan/.cargo/env
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
yarl 1.9.4 requires multidict>=4.0, which is not installed.
requests 2.31.0 requires charset-normalizer<4,>=2, which is not installed.
pygithub 1.58.2 requires deprecated, which is not installed.
posthog 3.5.0 requires backoff>=1.10.0, which is not installed.
posthog 3.5.0 requires monotonic>=1.5, which is not installed.
virtualenv 20.24.1 requires distlib<1,>=0.3.6, which is not installed.
requests-oauthlib 2.0.0 requires oauthlib>=3.0.0, which is not installed.
tokenizers 0.19.1 requires huggingface-hub<1.0,>=0.16.4, but you have huggingface-hub 1.1.4 which is incompatible.
virtualenv 20.24.1 requires platformdirs<4,>=3.5.1, but you have platformdirs 4.5.0 which is incompatible.

[notice] A new release of pip is availa

In [2]:
from google.colab import userdata, files
import os
from datetime import datetime
import numpy as np
from collections import defaultdict
import json
import tempfile
from pydub import AudioSegment
from speechbrain.inference import EncoderClassifier
from groq import Groq
from sklearn.cluster import AgglomerativeClustering
import shutil


/usr/local/lib/python3.12/dist-packages/pydub/utils.py:300: SyntaxWarning: invalid escape sequence '\('
  m = re.match('([su]([0-9]{1,2})p?) \(([0-9]{1,2}) bit\)$', token)
/usr/local/lib/python3.12/dist-packages/pydub/utils.py:301: SyntaxWarning: invalid escape sequence '\('
  m2 = re.match('([su]([0-9]{1,2})p?)( \(default\))?$', token)
/usr/local/lib/python3.12/dist-packages/pydub/utils.py:310: SyntaxWarning: invalid escape sequence '\('
  elif re.match('(flt)p?( \(default\))?$', token):
/usr/local/lib/python3.12/dist-packages/pydub/utils.py:314: SyntaxWarning: invalid escape sequence '\('
  elif re.match('(dbl)p?( \(default\))?$', token):
/usr/local/lib/python3.12/dist-packages/speechbrain/utils/torch_audio_backend.py:57: UserWarning: torchaudio._backend.list_audio_backends has been deprecated. This deprecation is part of a large refactoring effort to transition TorchAudio into a maintenance phase. The decoding and encoding capabilities of PyTorch for both audio and video are being c

In [3]:
try:
    GROQ_API_KEY = userdata.get('GROQ_API_KEY')
    os.environ['GROQ_API_KEY'] = GROQ_API_KEY
    print("✅ Groq API key loaded from Colab secrets")
except:
    from getpass import getpass
    GROQ_API_KEY = getpass("🔑 Enter your Groq API key: ")
    os.environ['GROQ_API_KEY'] = GROQ_API_KEY
    print("✅ Groq API key set manually")


🔑 Enter your Groq API key: ··········
✅ Groq API key set manually


In [4]:
print("📁 Upload your ENROLLMENT audio (your voice sample):")
uploaded = files.upload()
USER_AUDIO = list(uploaded.keys())[0]

print("\n📁 Upload your CONVERSATION audio (the audio to analyze):")
uploaded = files.upload()
CONVERSATION = list(uploaded.keys())[0]

print(f"\n✅ Files ready:")
print(f"   - Enrollment: {USER_AUDIO}")
print(f"   - Conversation: {CONVERSATION}")

# Create output folder
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
output_folder = f"segmented_audio_{timestamp}"
os.makedirs(output_folder, exist_ok=True)
print(f"\n📂 Output folder: {output_folder}")

📁 Upload your ENROLLMENT audio (your voice sample):


Saving enroll.mp3 to enroll.mp3

📁 Upload your CONVERSATION audio (the audio to analyze):


Saving test3_convo.mpeg to test3_convo.mpeg

✅ Files ready:
   - Enrollment: enroll.mp3
   - Conversation: test3_convo.mpeg

📂 Output folder: segmented_audio_20251115_201657


In [5]:
print("\n⚙️ Initializing models...")

# Initialize Groq client
groq_client = Groq(api_key=os.environ.get("GROQ_API_KEY"))
print("✅ Groq client initialized")

# Load SpeechBrain for speaker embeddings (CPU mode)
classifier = EncoderClassifier.from_hparams(
    source="speechbrain/spkrec-ecapa-voxceleb",
    run_opts={"device": "cpu"}
)
print("✅ SpeechBrain speaker recognition loaded")



⚙️ Initializing models...


INFO:speechbrain.utils.fetching:Fetch hyperparams.yaml: Fetching from HuggingFace Hub 'speechbrain/spkrec-ecapa-voxceleb' if not cached


✅ Groq client initialized


hyperparams.yaml: 0.00B [00:00, ?B/s]

INFO:speechbrain.utils.fetching:Fetch custom.py: Fetching from HuggingFace Hub 'speechbrain/spkrec-ecapa-voxceleb' if not cached
/usr/local/lib/python3.12/dist-packages/speechbrain/utils/torch_audio_backend.py:57: UserWarning: torchaudio._backend.list_audio_backends has been deprecated. This deprecation is part of a large refactoring effort to transition TorchAudio into a maintenance phase. The decoding and encoding capabilities of PyTorch for both audio and video are being consolidated into TorchCodec. Please see https://github.com/pytorch/audio/issues/3902 for more information. It will be removed from the 2.9 release. 
  available_backends = torchaudio.list_audio_backends()
DEBUG:speechbrain.utils.checkpoints:Registered checkpoint save hook for _save
DEBUG:speechbrain.utils.checkpoints:Registered checkpoint load hook for _load
DEBUG:speechbrain.utils.checkpoints:Registered parameter transfer hook for _load
/usr/local/lib/python3.12/dist-packages/speechbrain/utils/autocast.py:188: Fut

...

classifier.ckpt:   0%|          | 0.00/5.53M [00:00<?, ?B/s]

DEBUG:speechbrain.utils.parameter_transfer:Set local path in self.paths["classifier"] = /root/.cache/huggingface/hub/models--speechbrain--spkrec-ecapa-voxceleb/snapshots/0f99f2d0ebe89ac095bcc5903c4dd8f72b367286/classifier.ckpt
INFO:speechbrain.utils.fetching:Fetch label_encoder.txt: Fetching from HuggingFace Hub 'speechbrain/spkrec-ecapa-voxceleb' if not cached


label_encoder.txt: 0.00B [00:00, ?B/s]

DEBUG:speechbrain.utils.parameter_transfer:Set local path in self.paths["label_encoder"] = /root/.cache/huggingface/hub/models--speechbrain--spkrec-ecapa-voxceleb/snapshots/0f99f2d0ebe89ac095bcc5903c4dd8f72b367286/label_encoder.txt
INFO:speechbrain.utils.parameter_transfer:Loading pretrained files for: embedding_model, mean_var_norm_emb, classifier, label_encoder
DEBUG:speechbrain.utils.parameter_transfer:Redirecting (loading from local path): embedding_model -> /root/.cache/huggingface/hub/models--speechbrain--spkrec-ecapa-voxceleb/snapshots/0f99f2d0ebe89ac095bcc5903c4dd8f72b367286/embedding_model.ckpt
DEBUG:speechbrain.utils.parameter_transfer:Redirecting (loading from local path): mean_var_norm_emb -> /root/.cache/huggingface/hub/models--speechbrain--spkrec-ecapa-voxceleb/snapshots/0f99f2d0ebe89ac095bcc5903c4dd8f72b367286/mean_var_norm_emb.ckpt
DEBUG:speechbrain.utils.parameter_transfer:Redirecting (loading from local path): classifier -> /root/.cache/huggingface/hub/models--speechb

✅ SpeechBrain speaker recognition loaded


In [6]:
print("\n🎤 Extracting your voice embedding...")

user_signal = classifier.load_audio(USER_AUDIO)
user_embedding = classifier.encode_batch(user_signal).squeeze().detach().cpu().numpy()

print(f"✅ Voice embedding extracted: shape {user_embedding.shape}")



🎤 Extracting your voice embedding...


/usr/local/lib/python3.12/dist-packages/torchaudio/_backend/utils.py:213: UserWarning: In 2.9, this function's implementation will be changed to use torchaudio.load_with_torchcodec` under the hood. Some parameters like ``normalize``, ``format``, ``buffer_size``, and ``backend`` will be ignored. We recommend that you port your code to rely directly on TorchCodec's decoder instead: https://docs.pytorch.org/torchcodec/stable/generated/torchcodec.decoders.AudioDecoder.html#torchcodec.decoders.AudioDecoder.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torchaudio/_backend/ffmpeg.py:88: UserWarning: torio.io._streaming_media_decoder.StreamingMediaDecoder has been deprecated. This deprecation is part of a large refactoring effort to transition TorchAudio into a maintenance phase. The decoding and encoding capabilities of PyTorch for both audio and video are being consolidated into TorchCodec. Please see https://github.com/pytorch/audio/issues/3902 for more information. It will be r

✅ Voice embedding extracted: shape (192,)


In [7]:
print("\n🔄 Preparing audio for transcription...")

# Convert to WAV format
audio = AudioSegment.from_file(CONVERSATION)
wav_path = "temp_conversation.wav"

# Groq recommends 16kHz mono for best results
audio.set_channels(1).set_frame_rate(16000).export(wav_path, format="wav")

# Get audio duration
duration_seconds = len(audio) / 1000
print(f"✅ Audio prepared: {duration_seconds:.2f} seconds")

# Transcribe with Groq
print("\n📝 Transcribing with Groq Whisper (WORD-LEVEL)...")

with open(wav_path, "rb") as file:
    transcription = groq_client.audio.transcriptions.create(
        file=(wav_path, file.read()),
        model="whisper-large-v3-turbo",
        response_format="verbose_json",
        timestamp_granularities=["word", "segment"],
        language="en",
        temperature=0.0
    )

# Extract timestamps
words = transcription.words
segments = transcription.segments

print(f"✅ Transcription complete!")
print(f"   Total segments: {len(segments)}")
print(f"   Total words: {len(words)}")
print(f"   Processing time: ~{duration_seconds/216:.2f}s")



🔄 Preparing audio for transcription...
✅ Audio prepared: 50.05 seconds

📝 Transcribing with Groq Whisper (WORD-LEVEL)...
✅ Transcription complete!
   Total segments: 19
   Total words: 139
   Processing time: ~0.23s


In [8]:
print("\n🔍 Analyzing speaker for each WORD (maximum accuracy)...")
print("=" * 80)

matches = []
segment_files = []

for idx, word_data in enumerate(words):
    start = word_data['start']
    end = word_data['end']
    word = word_data['word'].strip()

    # Progress
    if idx % 10 == 0:
        print(f"Processing word {idx+1}/{len(words)}...", end="\r")

    # Extract audio segment
    seg_audio = audio[start * 1000 : end * 1000]

    # Skip very short segments
    if len(seg_audio) < 300:  # 0.3 seconds minimum
        continue

    # Calculate speaker embedding
    with tempfile.NamedTemporaryFile(suffix=".wav", delete=False) as tmp:
        seg_audio.export(tmp.name, format="wav")
        seg_signal = classifier.load_audio(tmp.name)
        seg_embedding = classifier.encode_batch(seg_signal).squeeze().detach().cpu().numpy()
        os.remove(tmp.name)

    # Calculate similarity
    similarity = 1 - np.dot(user_embedding, seg_embedding) / (
        np.linalg.norm(user_embedding) * np.linalg.norm(seg_embedding)
    )

    who = "USER" if similarity < 0.70 else "OTHER"

    # Save segment
    segment_filename = f"word_{idx+1:04d}_{who}_{start:.2f}-{end:.2f}.wav"
    segment_path = os.path.join(output_folder, segment_filename)
    seg_audio.export(segment_path, format="wav")

    matches.append((start, end, similarity, who, word))
    segment_files.append(segment_path)

print(f"\n✅ Word-level analysis complete: {len(matches)} words processed")



🔍 Analyzing speaker for each WORD (maximum accuracy)...


/usr/local/lib/python3.12/dist-packages/torchaudio/_backend/utils.py:213: UserWarning: In 2.9, this function's implementation will be changed to use torchaudio.load_with_torchcodec` under the hood. Some parameters like ``normalize``, ``format``, ``buffer_size``, and ``backend`` will be ignored. We recommend that you port your code to rely directly on TorchCodec's decoder instead: https://docs.pytorch.org/torchcodec/stable/generated/torchcodec.decoders.AudioDecoder.html#torchcodec.decoders.AudioDecoder.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torchaudio/_backend/ffmpeg.py:88: UserWarning: torio.io._streaming_media_decoder.StreamingMediaDecoder has been deprecated. This deprecation is part of a large refactoring effort to transition TorchAudio into a maintenance phase. The decoding and encoding capabilities of PyTorch for both audio and video are being consolidated into TorchCodec. Please see https://github.com/pytorch/audio/issues/3902 for more information. It will be r


✅ Word-level analysis complete: 56 words processed


In [9]:
print("\n🔍 Speaker Diarization - Production Ready...")
print("=" * 80)

# Step 1: Extract embeddings for all segments
print("Step 1: Extracting embeddings...")

segment_data = []

for idx, seg in enumerate(segments):
    start = seg['start']
    end = seg['end']
    text = seg['text'].strip()

    print(f"Processing {idx+1}/{len(segments)}...", end="\r")

    seg_audio = audio[start * 1000 : end * 1000]

    if len(seg_audio) < 500:
        continue

    with tempfile.NamedTemporaryFile(suffix=".wav", delete=False) as tmp:
        seg_audio.export(tmp.name, format="wav")
        seg_signal = classifier.load_audio(tmp.name)
        seg_embedding = classifier.encode_batch(seg_signal).squeeze().detach().cpu().numpy()
        os.remove(tmp.name)

    # Calculate similarity to USER
    similarity = 1 - np.dot(user_embedding, seg_embedding) / (
        np.linalg.norm(user_embedding) * np.linalg.norm(seg_embedding)
    )

    segment_data.append({
        'start': start,
        'end': end,
        'text': text,
        'embedding': seg_embedding,
        'similarity': float(similarity),
        'audio': seg_audio
    })

print(f"\n✅ Extracted {len(segment_data)} embeddings")

# Step 2: Smart USER Classification
print("\nStep 2: Classifying speakers (adaptive threshold)...")

# Calculate similarity statistics
similarities = [seg['similarity'] for seg in segment_data]
mean_sim = np.mean(similarities)
std_sim = np.std(similarities)

print(f"Similarity stats: mean={mean_sim:.3f}, std={std_sim:.3f}")

# Adaptive threshold
user_threshold = min(mean_sim + 0.5 * std_sim, 0.85)
print(f"Using adaptive threshold: {user_threshold:.3f}")

user_segments = []
other_segments = []

for seg in segment_data:
    if seg['similarity'] < user_threshold:
        seg['speaker'] = 'USER'
        user_segments.append(seg)
    else:
        other_segments.append(seg)

print(f"✅ USER: {len(user_segments)} segments")
print(f"✅ OTHER: {len(other_segments)} segments")

# Step 3: Cluster OTHER speakers with smart merging
if len(other_segments) > 1:
    print("\nStep 3: Clustering OTHER speakers...")

    other_embeddings = np.array([seg['embedding'] for seg in other_segments])

    # Initial clustering
    clustering = AgglomerativeClustering(
        n_clusters=None,
        distance_threshold=0.70,
        metric='cosine',
        linkage='average'
    )

    cluster_labels = list(clustering.fit_predict(other_embeddings))
    n_initial_clusters = len(set(cluster_labels))

    print(f"✅ Initial clusters: {n_initial_clusters}")

    # Smart merging
    cluster_counts = {}
    for label in cluster_labels:
        cluster_counts[label] = cluster_counts.get(label, 0) + 1

    small_clusters = [c for c, count in cluster_counts.items() if count <= 2]
    large_clusters = [c for c, count in cluster_counts.items() if count > 2]

    if small_clusters and large_clusters:
        print(f"🔄 Merging {len(small_clusters)} small cluster(s)...")

        for small_c in small_clusters:
            small_indices = [i for i, label in enumerate(cluster_labels) if label == small_c]
            small_embs = other_embeddings[small_indices]

            best_match = None
            best_distance = float('inf')

            for large_c in large_clusters:
                large_indices = [i for i, label in enumerate(cluster_labels) if label == large_c]
                large_embs = other_embeddings[large_indices]

                distances = []
                for small_emb in small_embs:
                    for large_emb in large_embs:
                        dist = 1 - np.dot(small_emb, large_emb) / (
                            np.linalg.norm(small_emb) * np.linalg.norm(large_emb)
                        )
                        distances.append(dist)
                avg_dist = np.mean(distances)

                if avg_dist < best_distance:
                    best_distance = avg_dist
                    best_match = large_c

            for i in small_indices:
                cluster_labels[i] = best_match

            print(f"   Merged cluster {small_c} → cluster {best_match} (distance: {best_distance:.3f})")

    # Renumber clusters
    unique_clusters = sorted(set(cluster_labels))
    cluster_mapping = {old: new for new, old in enumerate(unique_clusters)}
    cluster_labels = [cluster_mapping[label] for label in cluster_labels]

    n_final_speakers = len(set(cluster_labels))
    print(f"✅ Final other speaker(s): {n_final_speakers}")

    # Assign speaker labels
    for seg, cluster_id in zip(other_segments, cluster_labels):
        seg['speaker'] = f'Speaker {cluster_id + 1}'

elif len(other_segments) == 1:
    print("\nStep 3: Only 1 other speaker detected")
    other_segments[0]['speaker'] = 'Speaker 1'
else:
    print("\nStep 3: No other speakers found")

# Step 4: Combine and sort
all_segments = user_segments + other_segments
all_segments.sort(key=lambda x: x['start'])

# Step 5: Save segments
print("\nStep 4: Saving audio segments...")

matches = []
segment_files = []

for seg in all_segments:
    segment_filename = f"segment_{len(matches)+1:03d}_{seg['speaker'].replace(' ', '_')}_{seg['start']:.2f}-{seg['end']:.2f}.wav"
    segment_path = os.path.join(output_folder, segment_filename)
    seg['audio'].export(segment_path, format="wav")

    matches.append((seg['start'], seg['end'], seg['similarity'], seg['speaker'], seg['text']))
    segment_files.append(segment_path)

print(f"✅ Saved {len(matches)} audio segments")

# === DISPLAY RESULTS ===
print("\n📊 MULTI-SPEAKER IDENTIFICATION RESULTS:")
print("=" * 80)
print(f"{'SPEAKER':^12} | {'TIME':^20} | {'SIMILARITY':^12} | {'TEXT':^40}")
print("=" * 80)

speaker_counts = defaultdict(int)
speaker_times = defaultdict(float)

for start, end, sim, speaker, text in matches:
    speaker_counts[speaker] += 1
    speaker_times[speaker] += (end - start)

    time_str = f"{start:6.2f}s → {end:6.2f}s"
    sim_str = f"{sim:.3f}"
    text_preview = text[:40] + "..." if len(text) > 40 else text

    print(f"{speaker:^12} | {time_str:^20} | {sim_str:^12} | {text_preview}")

print("=" * 80)

# === SUMMARY ===
total_time = sum(speaker_times.values())

print(f"\n📈 SPEAKER SUMMARY:")
print("=" * 80)

for speaker in sorted(speaker_counts.keys()):
    count = speaker_counts[speaker]
    time = speaker_times[speaker]
    percentage = (time / total_time * 100) if total_time > 0 else 0

    print(f"{speaker:12} | Segments: {count:3d} | Time: {time:6.2f}s ({percentage:5.1f}%)")

print("=" * 80)
print(f"Total Speakers: {len(speaker_counts)}")
print(f"Total Segments: {len(matches)}")
print(f"Total Duration: {total_time:.2f}s")

# === SAVE METADATA & JSON ===
print("\n💾 Saving metadata...")

# Text metadata
metadata_path = os.path.join(output_folder, "multi_speaker_analysis.txt")

with open(metadata_path, 'w', encoding='utf-8') as f:
    f.write("🎙️ MULTI-SPEAKER DIARIZATION RESULTS (MAKI.AI)\n")
    f.write("=" * 100 + "\n")
    f.write(f"Processing Date: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
    f.write(f"Transcription: Groq Whisper Large v3 Turbo\n")
    f.write(f"Speaker Model: SpeechBrain ECAPA-TDNN\n")
    f.write(f"Clustering: Agglomerative with smart merging\n")
    f.write(f"Audio Duration: {duration_seconds:.2f} seconds\n")
    f.write(f"Total Speakers: {len(speaker_counts)}\n")
    f.write(f"Total Segments: {len(matches)}\n")
    f.write("=" * 100 + "\n\n")

    f.write("SPEAKER SUMMARY:\n")
    f.write("-" * 100 + "\n")
    for speaker in sorted(speaker_counts.keys()):
        count = speaker_counts[speaker]
        time = speaker_times[speaker]
        percentage = (time / total_time * 100) if total_time > 0 else 0
        f.write(f"{speaker:12} | Segments: {count:3d} | Time: {time:6.2f}s ({percentage:5.1f}%)\n")
    f.write("-" * 100 + "\n\n")

    f.write("DETAILED TRANSCRIPT:\n")
    f.write("-" * 100 + "\n")
    for idx, (start, end, sim, speaker, text) in enumerate(matches):
        f.write(f"\n[{idx+1:03d}] {speaker} ({start:.2f}s - {end:.2f}s) [similarity: {sim:.3f}]\n")
        f.write(f"{text}\n")
        f.write("-" * 100 + "\n")

print(f"✅ Metadata saved: {metadata_path}")

# JSON export
print("\n💾 Exporting to JSON...")

json_data = {
    "conversation_id": f"conv_{timestamp}",
    "metadata": {
        "processing_date": datetime.now().isoformat(),
        "audio_duration_seconds": float(duration_seconds),
        "transcription_model": "groq/whisper-large-v3-turbo",
        "speaker_model": "speechbrain/spkrec-ecapa-voxceleb",
        "total_speakers": len(speaker_counts),
        "total_segments": len(matches)
    },
    "speakers": {
        speaker: {
            "segment_count": int(speaker_counts[speaker]),
            "total_time_seconds": float(speaker_times[speaker]),
            "percentage": float((speaker_times[speaker] / total_time * 100) if total_time > 0 else 0),
            "is_user": speaker == "USER"
        }
        for speaker in sorted(speaker_counts.keys())
    },
    "segments": [
        {
            "segment_id": idx + 1,
            "start_time": float(start),
            "end_time": float(end),
            "duration": float(end - start),
            "speaker": speaker,
            "similarity_to_user": float(sim),
            "transcript": text,
            "audio_file": segment_files[idx] if idx < len(segment_files) else None
        }
        for idx, (start, end, sim, speaker, text) in enumerate(matches)
    ]
}

json_path = os.path.join(output_folder, "maki_conversation.json")
with open(json_path, 'w', encoding='utf-8') as f:
    json.dump(json_data, f, indent=2, ensure_ascii=False)

print(f"✅ JSON saved: {json_path}")

# === CREATE ZIP & DOWNLOAD ===
print(f"\n📦 Creating ZIP archive...")

zip_filename = f"{output_folder}.zip"
shutil.make_archive(output_folder, 'zip', output_folder)

print(f"✅ ZIP created: {zip_filename}")
print(f"\n⬇️ Downloading...")

files.download(zip_filename)

print("\n" + "=" * 80)
print("🎉 ALL DONE!")
print("=" * 80)
print(f"✅ Identified {len(speaker_counts)} speaker(s)")
print(f"✅ Processed {len(matches)} segments")
print(f"✅ Total duration: {total_time:.2f}s")
print(f"✅ Downloaded: {zip_filename}")
print("\n💡 Check your Downloads folder!")
print("=" * 80)


🔍 Speaker Diarization - Production Ready...
Step 1: Extracting embeddings...
Processing 19/19...
✅ Extracted 18 embeddings

Step 2: Classifying speakers (adaptive threshold)...
Similarity stats: mean=0.661, std=0.318
Using adaptive threshold: 0.820
✅ USER: 8 segments
✅ OTHER: 10 segments

Step 3: Clustering OTHER speakers...
✅ Initial clusters: 2
✅ Final other speaker(s): 2

Step 4: Saving audio segments...
✅ Saved 18 audio segments

📊 MULTI-SPEAKER IDENTIFICATION RESULTS:
  SPEAKER    |         TIME         |  SIMILARITY  |                   TEXT                  
    USER     |    0.00s →   2.36s   |    0.333     | Hey, thanks for meeting today. How was y...
 Speaker 1   |    3.32s →   6.86s   |    0.874     | Great. I finally went hiking at Mount Ra...
 Speaker 2   |    7.36s →  10.14s   |    1.001     | Nice. I spent it working on that client ...
    USER     |   10.68s →  12.76s   |    0.297     | I helped Tom with the technical sections...
    USER     |   13.48s →  18.54s   |  

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


🎉 ALL DONE!
✅ Identified 3 speaker(s)
✅ Processed 18 segments
✅ Total duration: 39.80s
✅ Downloaded: segmented_audio_20251115_201657.zip

💡 Check your Downloads folder!
